<section style="direction: ltr; text-align: center; margin: 30px;">

# **SQL-Fundamentals-Farsi**

</section>

<section style="direction: ltr; text-align: justyfy; margin: 30px;">

This project offers a versatile and automated solution for generating database reports in structured formats. By leveraging popular Python libraries such as Pandas, SQLAlchemy, and Openpyxl, it simplifies the process of extracting, displaying, and exporting data from SQL Server databases. With the added ability to handle both Excel and CSV outputs, it provides a flexible tool for data analysis and reporting needs.

</section>

---


### **Calling the SQL Server Connection Function**


In [4]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def create_connection(server, database, username=None, password=None):
    """
    Create a connection to the SQL Server database using SQLAlchemy.

    Parameters:
    - server: str, the name of the SQL Server
    - database: str, the name of the database
    - username: str, optional, SQL Server username (for SQL authentication)
    - password: str, optional, SQL Server password (for SQL authentication)

    Returns:
    - engine: sqlalchemy.engine.base.Engine, the SQLAlchemy engine object or None if the connection fails
    """
    try:
        if username and password:
            connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
        else:
            connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

        engine = create_engine(connection_string)

        # Test connection
        with engine.connect() as conn:
            logging.info("Connection successful!")

        return engine

    except SQLAlchemyError as e:
        logging.error(f"Connection failed: {e}")
        return None

---

### **Check if the database connection is successful**


In [5]:
def check_connection(engine):
    if engine:
        logging.info("Database connection established successfully.")
        logging.info(f"Server: {server}")
        logging.info(f"Database: {database}")
        return True
    else:
        logging.error("Failed to establish database connection.")
        return False


# Call the create_connection function
server = r"imsalione-pc\imsalionedb"
database = "pubs"

engine = create_connection(server, database)

# Use the check_connection function to verify the connection
if check_connection(engine):
    # Proceed with database operations
    pass
else:
    # Handle the failed connection
    pass

2024-10-21 17:48:45,732 - INFO - Connection successful!
2024-10-21 17:48:45,733 - INFO - Database connection established successfully.
2024-10-21 17:48:45,734 - INFO - Server: imsalione-pc\imsalionedb
2024-10-21 17:48:45,734 - INFO - Database: pubs


---

### **Fetching and Displaying Data from SQL Server**


In [6]:
import pandas as pd
from sqlalchemy import exc
from pathlib import Path
from prettytable import PrettyTable

# Define the directory where you want to save the report files
output_directory = Path("C:/Reports")  # Adjust the path as needed
output_directory.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists


def truncate(value, width):
    return str(value)[:width] + "..." if len(str(value)) > width else str(value)


def fetch_and_display_queries(
    engine,
    queries,
    pnum=10,
    export_to_excel=False,
    export_to_csv=False,
    display_tail=False,
):
    """
    Fetch data from multiple SQL queries and optionally export to Excel or CSV.

    Parameters:
    - engine: SQLAlchemy engine object.
    - queries: List of SQL query strings.
    - pnum: int, number of rows to display (default is 10).
    - export_to_excel: bool, whether to export data to an Excel file (default is False).
    - export_to_csv: bool, whether to export data to a CSV file (default is False).
    - display_tail: bool, whether to display the last pnum rows (default is False).
    """

    if engine:
        # If exporting to Excel, create the writer object for the single file
        if export_to_excel:
            file_path = (
                output_directory
                / f"report_{pd.Timestamp.now().strftime('%d%m%Y%H%M%S')}.xlsx"
            )
            writer = pd.ExcelWriter(file_path, engine="openpyxl")

        for i, query in enumerate(queries, 1):

            print(f"Executing query {i}:\n{query}")
            try:
                # Load data into a pandas DataFrame
                df = pd.read_sql(query, engine)

                # Display the first or last pnum rows based on display_tail parameter
                table = PrettyTable()
                table.field_names = df.columns

                if display_tail:
                    rows = df.iloc[-pnum:].itertuples(index=False)
                else:
                    rows = df.iloc[:pnum].itertuples(index=False)

                for row in rows:
                    table.add_row([truncate(value, 25) for value in row])

                print(table)

                # Print the number of records in the query result
                record_count = len(df)
                print(f"Query {i} returned {record_count} records.\n")

                # Export to Excel in different sheets if requested
                if export_to_excel:
                    sheet_name = f"Query_{i}"
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(
                        f"Data from query {i} exported to sheet {sheet_name} in {file_path}"
                    )

                # Export to CSV if requested (each query in a different CSV file)
                if export_to_csv:
                    csv_file_path = (
                        output_directory
                        / f"report_{i}_{pd.Timestamp.now().strftime('%d%m%Y%H%M%S')}.csv"
                    )
                    df.to_csv(csv_file_path, index=False)
                    print(f"Data from query {i} exported to CSV at {csv_file_path}")

            except exc.SQLAlchemyError as db_error:
                print(f"Database error occurred during query {i}: {db_error}")
            except Exception as e:
                print(f"An error occurred during query {i}: {e}")

        # Save the Excel file with all sheets after the loop finishes
        if export_to_excel:
            writer.close()  # Use close() instead of save()
            print(f"All data exported to {file_path}")

    else:
        print("No database connection.")


# Example usage:
# queries = ["""SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Customer';""", ...]
# fetch_and_display_multiple_queries(engine, queries, pnum=20, export_to_excel=True, export_to_csv=True, sidplay_tail=True)

---

### **Executing SQL Query to Fetch and Display Table Information**


In [7]:
queries = [
    """SELECT TABLE_SCHEMA, TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES"""
]

fetch_and_display_queries(engine, queries, pnum=20)

Executing query 1:
SELECT TABLE_SCHEMA, TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES
+--------------+-------------+
| TABLE_SCHEMA |  TABLE_NAME |
+--------------+-------------+
|     dbo      | sysdiagrams |
|     dbo      |   authors   |
|     dbo      |  publishers |
|     dbo      |    titles   |
|     dbo      | titleauthor |
|     dbo      |    stores   |
|     dbo      |   roysched  |
|     dbo      |  discounts  |
|     dbo      |     jobs    |
|     dbo      |   pub_info  |
|     dbo      |   employee  |
|     dbo      |  titleview  |
|     dbo      |    sales    |
+--------------+-------------+
Query 1 returned 13 records.



In [8]:
queries = [
    """SELECT au_fname + ' ' + au_lname as N'نام کامل'
           FROM dbo.authors"""
]

fetch_and_display_queries(engine, queries, pnum=10)

Executing query 1:
SELECT au_fname + ' ' + au_lname as N'نام کامل'
           FROM dbo.authors
+-------------------------+
|         نام کامل        |
+-------------------------+
|      Abraham Bennet     |
| Reginald Blotchet-Halls |
|      Cheryl Carson      |
|     Michel DeFrance     |
|    Innes del Castillo   |
|         Ann Dull        |
|      Marjorie Green     |
|    Morningstar Greene   |
|     Burt Gringlesby     |
|      Sheryl Hunter      |
+-------------------------+
Query 1 returned 23 records.



In [9]:
query = ["""SELECT fname, lname, job_id FROM employee"""]

fetch_and_display_queries(engine, query, 10)

Executing query 1:
SELECT fname, lname, job_id FROM employee
+-----------+-----------+--------+
|   fname   |   lname   | job_id |
+-----------+-----------+--------+
|   Paolo   |  Accorti  |   13   |
|   Pedro   |   Afonso  |   14   |
|  Victoria |  Ashworth |   6    |
|   Helen   |  Bennett  |   12   |
|   Lesley  |   Brown   |   7    |
| Francisco |   Chang   |   4    |
|   Philip  |   Cramer  |   2    |
|    Aria   |    Cruz   |   10   |
|    Ann    |   Devon   |   3    |
|  Anabela  | Domingues |   8    |
+-----------+-----------+--------+
Query 1 returned 43 records.



In [10]:
query = [
    """SELECT emp_id, fname + ' ' + lname AS FullName, hire_date
         FROM dbo.employee
         ORDER BY hire_date DESC"""
]

fetch_and_display_queries(engine, query, pnum=5)

Executing query 1:
SELECT emp_id, fname + ' ' + lname AS FullName, hire_date
         FROM dbo.employee
         ORDER BY hire_date DESC
+-----------+------------------+---------------------+
|   emp_id  |     FullName     |      hire_date      |
+-----------+------------------+---------------------+
| MGK44605M | Matti Karttunen  | 1994-05-01 00:00:00 |
| KJJ92907F | Karla Jablonski  | 1994-03-11 00:00:00 |
| PSP68661F |  Paula Parente   | 1994-01-19 00:00:00 |
| POK93028M | Pirkko Koskitalo | 1993-11-29 00:00:00 |
| RBM23061F |   Rita Muller    | 1993-10-09 00:00:00 |
+-----------+------------------+---------------------+
Query 1 returned 43 records.



In [11]:
query = [
    """SELECT emp_id, fname + ' ' + lname AS FullName, MONTH(hire_date) AS MonthHireDate
         FROM dbo.employee
         ORDER BY MONTH(hire_date) DESC
"""
]

fetch_and_display_queries(engine, query, pnum=10)

Executing query 1:
SELECT emp_id, fname + ' ' + lname AS FullName, MONTH(hire_date) AS MonthHireDate
         FROM dbo.employee
         ORDER BY MONTH(hire_date) DESC

+-----------+------------------+---------------+
|   emp_id  |     FullName     | MonthHireDate |
+-----------+------------------+---------------+
| PSA89086M |   Pedro Afonso   |       12      |
| MAP77183M |  Miguel Paolino  |       12      |
| DWR65030M |    Diego Roel    |       12      |
| POK93028M | Pirkko Koskitalo |       11      |
| F-C16315M | Francisco Chang  |       11      |
| PTC11962M |  Philip Cramer   |       11      |
| HAS54740M |  Howard Snyder   |       11      |
| A-C71970F |    Aria Cruz     |       10      |
| KFJ64308F |  Karin Josephs   |       10      |
| RBM23061F |   Rita Muller    |       10      |
+-----------+------------------+---------------+
Query 1 returned 43 records.



In [12]:
query = [
    """SELECT * FROM publishers
         WHERE  country = 'usa'"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT * FROM publishers
         WHERE  country = 'usa'
+--------+-----------------------+------------+-------+---------+
| pub_id |        pub_name       |    city    | state | country |
+--------+-----------------------+------------+-------+---------+
|  0736  |     New Moon Books    |   Boston   |   MA  |   USA   |
|  0877  |    Binnet & Hardley   | Washington |   DC  |   USA   |
|  1389  |  Algodata Infosystems |  Berkeley  |   CA  |   USA   |
|  1622  | Five Lakes Publishing |  Chicago   |   IL  |   USA   |
|  1756  |   Ramona Publishers   |   Dallas   |   TX  |   USA   |
|  9952  |     Scootney Books    |  New York  |   NY  |   USA   |
+--------+-----------------------+------------+-------+---------+
Query 1 returned 6 records.



In [13]:
query = [
    """SELECT * FROM publishers
         WHERE country in ('Germany', 'France')"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT * FROM publishers
         WHERE country in ('Germany', 'France')
+--------+--------------------+---------+-------+---------+
| pub_id |      pub_name      |   city  | state | country |
+--------+--------------------+---------+-------+---------+
|  9901  |       GGG&G        | M?nchen |  None | Germany |
|  9999  | Lucerne Publishing |  Paris  |  None |  France |
+--------+--------------------+---------+-------+---------+
Query 1 returned 2 records.



In [14]:
query = [
    """SELECT * FROM sales
         WHERE YEAR(ord_date) = 1993"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT * FROM sales
         WHERE YEAR(ord_date) = 1993
+----+---------+---------+---------------------+-----+------------+----------+
| id | stor_id | ord_num |       ord_date      | qty |  payterms  | title_id |
+----+---------+---------+---------------------+-----+------------+----------+
| 3  |   7066  |  A2976  | 1993-05-24 00:00:00 |  50 |   Net 30   |  PC8888  |
| 11 |   7131  |  P3087a | 1993-05-29 00:00:00 |  20 |   Net 60   |  PS1372  |
| 12 |   7131  |  P3087a | 1993-05-29 00:00:00 |  25 |   Net 60   |  PS2106  |
| 13 |   7131  |  P3087a | 1993-05-29 00:00:00 |  15 |   Net 60   |  PS3333  |
| 14 |   7131  |  P3087a | 1993-05-29 00:00:00 |  25 |   Net 60   |  PS7777  |
| 15 |   7896  |  QQ2299 | 1993-10-28 00:00:00 |  15 |   Net 60   |  BU7832  |
| 16 |   7896  |  TQ456  | 1993-12-12 00:00:00 |  10 |   Net 60   |  MC2222  |
| 17 |   7896  |   X999  | 1993-02-21 00:00:00 |  35 | ON invoice |  BU2075  |
| 20 |   8042  |   P723  | 1993-03-11 00:00:00 |  25 | 

In [15]:
query = [
    """SELECT *, CASE WHEN qty < 30 THEN N'کم فروش'
         WHEN qty >= 30 THEN N'پر فروش'
         END AS SalesClass
         FROM Sales"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT *, CASE WHEN qty < 30 THEN N'کم فروش'
         WHEN qty >= 30 THEN N'پر فروش'
         END AS SalesClass
         FROM Sales
+----+---------+----------+---------------------+-----+------------+----------+------------+
| id | stor_id | ord_num  |       ord_date      | qty |  payterms  | title_id | SalesClass |
+----+---------+----------+---------------------+-----+------------+----------+------------+
| 1  |   6380  |   6871   | 1994-09-14 00:00:00 |  5  |   Net 60   |  BU1032  |  کم فروش   |
| 2  |   6380  |   722a   | 1994-09-13 00:00:00 |  3  |   Net 60   |  PS2091  |  کم فروش   |
| 3  |   7066  |  A2976   | 1993-05-24 00:00:00 |  50 |   Net 30   |  PC8888  |  پر فروش   |
| 4  |   7066  | QA7442.3 | 1994-09-13 00:00:00 |  75 | ON invoice |  PS2091  |  پر فروش   |
| 5  |   7067  |  D4482   | 1994-09-14 00:00:00 |  10 |   Net 60   |  PS2091  |  کم فروش   |
| 6  |   7067  |  P2121   | 1992-06-15 00:00:00 |  40 |   Net 30   |  TC3218  |  پر فروش   |
| 7  |   706

In [16]:
query = [
    """SELECT emp_id, fname, lname, DATEDIFF(year, hire_date, GETDATE()) 
         AS Experience
         FROM employee"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT emp_id, fname, lname, DATEDIFF(year, hire_date, GETDATE()) 
         AS Experience
         FROM employee
+-----------+-----------+-----------+------------+
|   emp_id  |   fname   |   lname   | Experience |
+-----------+-----------+-----------+------------+
| PMA42628M |   Paolo   |  Accorti  |     32     |
| PSA89086M |   Pedro   |   Afonso  |     34     |
| VPA30890F |  Victoria |  Ashworth |     34     |
| H-B39728F |   Helen   |  Bennett  |     35     |
| L-B31947F |   Lesley  |   Brown   |     33     |
| F-C16315M | Francisco |   Chang   |     34     |
| PTC11962M |   Philip  |   Cramer  |     35     |
| A-C71970F |    Aria   |    Cruz   |     33     |
| AMD15433F |    Ann    |   Devon   |     33     |
| ARD36773F |  Anabela  | Domingues |     31     |
+-----------+-----------+-----------+------------+
Query 1 returned 43 records.



In [17]:
query = [
    """SELECT emp_id, fname, lname, DATEDIFF(YEAR, hire_date, GETDATE())
         AS Experience,
         CASE WHEN DATEDIFF(YEAR, hire_date, GETDATE()) > 30 THEN 'G1'
         WHEN DATEDIFF(YEAR, hire_date, GETDATE()) <= 30 THEN 'G2'
         END AS ExperinceClass
         FROM employee"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT emp_id, fname, lname, DATEDIFF(YEAR, hire_date, GETDATE())
         AS Experience,
         CASE WHEN DATEDIFF(YEAR, hire_date, GETDATE()) > 30 THEN 'G1'
         WHEN DATEDIFF(YEAR, hire_date, GETDATE()) <= 30 THEN 'G2'
         END AS ExperinceClass
         FROM employee
+-----------+-----------+-----------+------------+----------------+
|   emp_id  |   fname   |   lname   | Experience | ExperinceClass |
+-----------+-----------+-----------+------------+----------------+
| PMA42628M |   Paolo   |  Accorti  |     32     |       G1       |
| PSA89086M |   Pedro   |   Afonso  |     34     |       G1       |
| VPA30890F |  Victoria |  Ashworth |     34     |       G1       |
| H-B39728F |   Helen   |  Bennett  |     35     |       G1       |
| L-B31947F |   Lesley  |   Brown   |     33     |       G1       |
| F-C16315M | Francisco |   Chang   |     34     |       G1       |
| PTC11962M |   Philip  |   Cramer  |     35     |       G1       |
| A-C71970F |    Ar

In [18]:
query = [
    """SELECT * FROM sales
         WHERE title_id LIKE 'B%'"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT * FROM sales
         WHERE title_id LIKE 'B%'
+----+---------+----------+---------------------+-----+------------+----------+
| id | stor_id | ord_num  |       ord_date      | qty |  payterms  | title_id |
+----+---------+----------+---------------------+-----+------------+----------+
| 1  |   6380  |   6871   | 1994-09-14 00:00:00 |  5  |   Net 60   |  BU1032  |
| 15 |   7896  |  QQ2299  | 1993-10-28 00:00:00 |  15 |   Net 60   |  BU7832  |
| 17 |   7896  |   X999   | 1993-02-21 00:00:00 |  35 | ON invoice |  BU2075  |
| 19 |   8042  | 423LL930 | 1994-09-14 00:00:00 |  10 | ON invoice |  BU1032  |
| 20 |   8042  |   P723   | 1993-03-11 00:00:00 |  25 |   Net 30   |  BU1111  |
+----+---------+----------+---------------------+-----+------------+----------+
Query 1 returned 5 records.



In [19]:
query = [
    """SELECT title, type, price FROM titles
         WHERE price IS NULL"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT title, type, price FROM titles
         WHERE price IS NULL
+------------------------------+--------------+-------+
|            title             |     type     | price |
+------------------------------+--------------+-------+
| The Psychology of Compute... | UNDECIDED    |  None |
|        Net Etiquette         | popular_comp |  None |
+------------------------------+--------------+-------+
Query 1 returned 2 records.



In [20]:
query = [
    """SELECT title, type, ISNULL(price, 0) AS NewPrice
         FROM titles
         ORDER BY NewPrice DESC"""
]

fetch_and_display_queries(engine, query, pnum=5, display_tail=True)

Executing query 1:
SELECT title, type, ISNULL(price, 0) AS NewPrice
         FROM titles
         ORDER BY NewPrice DESC
+------------------------------+--------------+----------+
|            title             |     type     | NewPrice |
+------------------------------+--------------+----------+
|      Life Without Fear       | psychology   |   7.0    |
| You Can Combat Computer S... | business     |   2.99   |
|    The Gourmet Microwave     | mod_cook     |   2.99   |
| The Psychology of Compute... | UNDECIDED    |   0.0    |
|        Net Etiquette         | popular_comp |   0.0    |
+------------------------------+--------------+----------+
Query 1 returned 18 records.



---

### 15- Three books with the highest price


In [21]:
query = [
    """SELECT TOP 3 title, type, price
         FROM titles
         ORDER BY price DESC"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT TOP 3 title, type, price
         FROM titles
         ORDER BY price DESC
+------------------------------+--------------+-------+
|            title             |     type     | price |
+------------------------------+--------------+-------+
|   But Is It User Friendly?   | popular_comp | 22.95 |
| Computer Phobic AND Non-P... | psychology   | 21.59 |
| Onions, Leeks, and Garlic... | trad_cook    | 20.95 |
+------------------------------+--------------+-------+
Query 1 returned 3 records.



---

### 16- Three newly hired employees


In [22]:
query = [
    """SELECT TOP 3 fname, lname
         FROM employee
         ORDER BY hire_date DESC"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT TOP 3 fname, lname
         FROM employee
         ORDER BY hire_date DESC
+-------+-----------+
| fname |   lname   |
+-------+-----------+
| Matti | Karttunen |
| Karla | Jablonski |
| Paula |  Parente  |
+-------+-----------+
Query 1 returned 3 records.



---

### 17- Books whose price is in a certain range


In [23]:
query = [
    """SELECT title, type, price 
         FROM titles
         WHERE price between 7 and 20"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT title, type, price 
         FROM titles
         WHERE price between 7 and 20
+------------------------------+--------------+-------+
|            title             |     type     | price |
+------------------------------+--------------+-------+
| The Busy Executive's Data... | business     | 19.99 |
| Cooking with Computers: S... | business     | 11.95 |
| Straight Talk About Compu... | business     | 19.99 |
| Silicon Valley Gastronomi... | mod_cook     | 19.99 |
|  Secrets of Silicon Valley   | popular_comp |  20.0 |
|     Is Anger the Enemy?      | psychology   | 10.95 |
|      Life Without Fear       | psychology   |  7.0  |
| Prolonged Data Deprivatio... | psychology   | 19.99 |
| Emotional Security: A New... | psychology   |  7.99 |
| Fifty Years in Buckingham... | trad_cook    | 11.95 |
+------------------------------+--------------+-------+
Query 1 returned 11 records.



---

### 18- Books published after a certain year


In [24]:
query = [
    """SELECT title, type, YEAR(pubdate) AS PublishedDate
         FROM titles
         WHERE YEAR(pubdate) > 1991"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT title, type, YEAR(pubdate) AS PublishedDate
         FROM titles
         WHERE YEAR(pubdate) > 1991
+------------------------------+--------------+---------------+
|            title             |     type     | PublishedDate |
+------------------------------+--------------+---------------+
| The Psychology of Compute... | UNDECIDED    |      2023     |
|  Secrets of Silicon Valley   | popular_comp |      1994     |
|        Net Etiquette         | popular_comp |      2023     |
+------------------------------+--------------+---------------+
Query 1 returned 3 records.



---

### 19- Calculation operation on the price of books


In [25]:
query = [
    """SELECT SUM(price) AS SumPrice,
         AVG(price) AS AVGPrice,
         MAX(price) AS MAXPrice,
         MIN(price) AS MINPrice
         FROM titles"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT SUM(price) AS SumPrice,
         AVG(price) AS AVGPrice,
         MAX(price) AS MAXPrice,
         MIN(price) AS MINPrice
         FROM titles
+----------+----------+----------+----------+
| SumPrice | AVGPrice | MAXPrice | MINPrice |
+----------+----------+----------+----------+
|  236.26  | 14.7662  |  22.95   |   2.99   |
+----------+----------+----------+----------+
Query 1 returned 1 records.



---

### 20- Calculate the total number of authors


In [26]:
query = [
    """SELECT COUNT(*) AS NumberOfAuthors
         FROM authors"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT COUNT(*) AS NumberOfAuthors
         FROM authors
+-----------------+
| NumberOfAuthors |
+-----------------+
|        23       |
+-----------------+
Query 1 returned 1 records.



---

### 21- Average calculation for each book category


In [27]:
query = [
    """SELECT type, AVG(price) AS PriceAvege
         FROM titles
         GROUP BY type"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT type, AVG(price) AS PriceAvege
         FROM titles
         GROUP BY type
+--------------+------------+
|     type     | PriceAvege |
+--------------+------------+
| business     |   13.73    |
| mod_cook     |   11.49    |
| popular_comp |   21.475   |
| psychology   |   13.504   |
| trad_cook    |  15.9633   |
| UNDECIDED    |    nan     |
+--------------+------------+
Query 1 returned 6 records.



---

### 22- The number of employed people by year


In [29]:
query = [
    """SELECT COUNT(*) AS NumberOfHiredEmployee,
         YEAR(hire_date) AS YearOfHiring
         FROM employee
         GROUP BY YEAR(hire_date)
         ORDER BY NumberOfHiredEmployee DESC"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT COUNT(*) AS NumberOfHiredEmployee,
         YEAR(hire_date) AS YearOfHiring
         FROM employee
         GROUP BY YEAR(hire_date)
         ORDER BY NumberOfHiredEmployee DESC
+-----------------------+--------------+
| NumberOfHiredEmployee | YearOfHiring |
+-----------------------+--------------+
|           8           |     1990     |
|           7           |     1991     |
|           7           |     1992     |
|           7           |     1993     |
|           7           |     1989     |
|           4           |     1988     |
|           3           |     1994     |
+-----------------------+--------------+
Query 1 returned 7 records.



---

### 23- Calculation of years with a certain number of employees


In [31]:
query = [
    """SELECT COUNT(*) AS NumberOfHiredEmployee,
         YEAR(hire_date) AS YEAROfHiring
         FROM employee
         GROUP BY YEAR(hire_date)
         HAVING COUNT(*) > 5
         ORDER BY NumberOfHiredEmployee DESC"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT COUNT(*) AS NumberOfHiredEmployee,
         YEAR(hire_date) AS YEAROfHiring
         FROM employee
         GROUP BY YEAR(hire_date)
         HAVING COUNT(*) > 5
         ORDER BY NumberOfHiredEmployee DESC
+-----------------------+--------------+
| NumberOfHiredEmployee | YEAROfHiring |
+-----------------------+--------------+
|           8           |     1990     |
|           7           |     1991     |
|           7           |     1992     |
|           7           |     1993     |
|           7           |     1989     |
+-----------------------+--------------+
Query 1 returned 5 records.



---

### 24- Number of authors by state


In [33]:
query = [
    """SELECT COUNT(*) AS NumberOfAuthors,
         state
         FROM authors
         GROUP BY state
         ORDER BY NumberOfAuthors DESC"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT COUNT(*) AS NumberOfAuthors,
         state
         FROM authors
         GROUP BY state
         ORDER BY NumberOfAuthors DESC
+-----------------+-------+
| NumberOfAuthors | state |
+-----------------+-------+
|        15       |   CA  |
|        2        |   UT  |
|        1        |   IN  |
|        1        |   KS  |
|        1        |   MD  |
|        1        |   MI  |
|        1        |   OR  |
|        1        |   TN  |
+-----------------+-------+
Query 1 returned 8 records.



---

### 25- Average price of books by year


In [37]:
query = [
    """SELECT YEAR(pubdate) AS PubkicationYear,
         AVG(price) AS AveragePrice
         FROM titles
         GROUP BY YEAR(pubdate)"""
]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT YEAR(pubdate) AS PubkicationYear,
         AVG(price) AS AveragePrice
         FROM titles
         GROUP BY YEAR(pubdate)
+-----------------+--------------+
| PubkicationYear | AveragePrice |
+-----------------+--------------+
|       1991      |   14.4173    |
|       1994      |     20.0     |
|       2023      |     nan      |
+-----------------+--------------+
Query 1 returned 3 records.



---

### 26- List of unique cities in the database


In [36]:
query = ["""SELECT DISTINCT city FROM authors"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT DISTINCT city FROM authors
+------------+
|    city    |
+------------+
| Ann Arbor  |
|  Berkeley  |
| Corvallis  |
|   Covelo   |
|    Gary    |
|  Lawrence  |
| Menlo Park |
| Nashville  |
|  Oakland   |
| Palo Alto  |
+------------+
Query 1 returned 16 records.

